# Skelettschätzer

In [1]:
!pip install ipywidgets tomli ray
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

import os
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, clear_output, HTML
import ray
import subprocess
import yaml
import uuid
ray.init()

Defaulting to user installation because normal site-packages is not writeable
(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
/home/fese4852/.local/lib/python3.8/site-packages/jupyterlab/debuglog.py:54: UserWarning: An error occurred.
  warnings.warn("An error occurred.")
/home/fese4852/.local/lib/python3.8/site-packages/jupyterlab/debuglog.py:55: UserWarning: ValueError: Please install Node.js and npm before continuing installation. You may be able to install Node.js from your package manager, from conda, or directly from the Node.js website (https://nodejs.org).
  warnings.warn(msg[-1].strip())
/home/fese4852/.local/lib/python3.8/site-packages/jupyterlab/debuglog.py:56: UserWarning: See the log file for 

2025-06-15 19:13:51,425	INFO worker.py:1752 -- Started a local Ray instance.


Python version:,3.8.10
Ray version:,2.10.0


(run_container_as_service pid=657026) ⏳ Docker-Container läuft: cpm
(run_container_as_service pid=657018) ⏳ Docker-Container läuft: tfpose
(run_container_as_service pid=657022) ⏳ Docker-Container läuft: hrnetv2
(run_container_as_service pid=657024) ⏳ Docker-Container läuft: dsnt
(run_container_as_service pid=657028) ⏳ Docker-Container läuft: simcc
(run_container_as_service pid=657017) ⏳ Docker-Container läuft: densepose
(run_container_as_service pid=657021) ⏳ Docker-Container läuft: hrnet
(run_container_as_service pid=657023) ⏳ Docker-Container läuft: ipr
(run_container_as_service pid=657031) ⏳ Docker-Container läuft: hourglass
(run_container_as_service pid=657012) ⏳ Docker-Container läuft: debias
(run_container_as_service pid=657016) ⏳ Docker-Container läuft: rsn
(run_container_as_service pid=657030) ⏳ Docker-Container läuft: higher-hrnet
(run_container_as_service pid=657020) ⏳ Docker-Container läuft: mspn
(run_container_as_service pid=657015) ⏳ Docker-Container läuft: scnet
(run_cont

(run_container_as_service pid=657019) time="2025-06-15T19:14:14+02:00" level=warning msg="/home/fese4852/Skelettschaetzer/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
(run_container_as_service pid=657025) time="2025-06-15T19:14:14+02:00" level=warning msg="Found orphan containers ([input-loader-0e6fd118 codecs-f581bff5 hourglass-4acd65cd deeppose-888f45dd rsn-f0f73809 higher-hrnet-105a4e32 densepose-fe578913 scnet-49d279f7 hrnet-aec18caf blazepose-7b43fac3 litehrnet-25def32d simplebaseline2d-7e0a9be9 mspn-bc76b530 lightweight-95b440d2 cpm-c72021df hrnetv2-8aff66dd ipr-10ef4a52 simcc-2d6b2325 debias-1d208eef dsnt-fa9f2c2b tfpose-2f234962 vipnas-a205fc88 input-loader-555d5161 densepose-2f2a9432 hrnet-c0846b5b simcc-37890154 rsn-18bd8cd1 cpm-cb726911 deeppose-250bf1d7 simplebaseline2d-c258aa00 dsnt-f856c3bc blazepose-0d9dc882 litehrnet-9e02e66a mspn-17e02a42 scnet-ba7be25c debias-60a802f4 hourglass-7955a288 ipr

...

(run_container_as_service pid=657028) 06/15 17:21:38 - mmengine - WARNING - Failed to search registry with scope "mmpose" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpose" is a correct scope, or whether the registry is initialized.
(run_container_as_service pid=657024) Loads checkpoint by local backend from path: simcc_res50_8xb32-140e_coco-384x288-45c3ba34_20220913.pth [repeated 9x across cluster]
(run_container_as_service pid=657016) 06/15 17:21:38 - mmengine - WARNING - dataset_meta are not saved in the checkpoint's meta data, load via config. [repeated 8x across cluster]


(run_container_as_service pid=657023) Traceback (most recent call last):
(run_container_as_service pid=657023)   File "exec.py", line 17, in <module>
(run_container_as_service pid=657023)   File "/app/mmpose/mmpose/apis/inferencers/mmpose_inferencer.py", line 102, in __init__
(run_container_as_service pid=657023)     det_cat_ids, show_progress)
(run_container_as_service pid=657023)   File "/app/mmpose/mmpose/apis/inferencers/pose2d_inferencer.py", line 91, in __init__
(run_container_as_service pid=657023)     show_progress=show_progress)
(run_container_as_service pid=657023)   File "/app/mmpose/mmpose/apis/inferencers/base_mmpose_inferencer.py", line 65, in __init__
(run_container_as_service pid=657023)     model, weights, device, scope, show_progress=show_progress)
(run_container_as_service pid=657023)   File "/opt/conda/lib/python3.7/site-packages/mmengine/infer/infer.py", line 180, in __init__
(run_container_as_service pid=657023)     self.model = self._init_model(cfg, weights, devi

(run_container_as_service pid=657020) load checkpoint from local path: ./higher_hrnet32_coco_512x512-8ae85183_20200713.pth


(run_container_as_service pid=657021)     first_frame = cv2.imread(image_paths[0])
(run_container_as_service pid=657021) IndexError: list index out of range


(run_container_as_service pid=657015) ❌ Fehler beim Ausführen von: deeppose (Exit-Code: 1)
(run_container_as_service pid=657015) 📜 Container-Logs von deeppose-4e71856e:


ℹ Bitte aktivieren Sie die Logs, denn einige Ausgaben können nur darüber angezeigt werden.
`View > Show Log Console`

## 1. Auswählen von Videos
Wählen Sie zunächst Videos aus, die verarbeitet werden. Sie können mp4-Dateien ins Arbeitsverzeichnis laden. Außerdem können Sie YouTube-Videos per Link einfügen, die dann verarbeitet werden.

In [2]:
# Data-Ordner finden
with open("./.env", "r") as f:
    env = [line.replace("\n", "").split("=") for line in f.readlines()]
env = {e[0]: e[1] for e in env}
data_folder = env["INPUT_FOLDER"]
input_folder = os.path.join(data_folder, "input")
input_folder_parsed = Path(input_folder)
# YouTube-Videos suchen
if "input.txt" in os.listdir(input_folder):
    with open(os.path.join(input_folder, "input.txt"), "r") as f:
        yt_links = f.read()
else:
    yt_links = ""

# ---------- Widgets ----------

# Datei-Upload (nur mp4 zulassen)
upload_widget = widgets.FileUpload(
  accept='.mp4',
  multiple=True,
  description='MP4 hochladen'
)

# Mehrzeiliges Eingabefeld für URLs
url_textarea = widgets.Textarea(
  placeholder='Gib hier YouTube-Links ein (einer pro Zeile)',
  description='URLs:',
  layout=widgets.Layout(width='100%', height='100px'),
  value=yt_links
)

# Button zum Löschen aller MP4-Dateien
delete_button = widgets.Button(
  description='Alle MP4-Dateien löschen',
  button_style='danger'
)

# Bestätigungsbutton
confirm_button = widgets.Button(
  description='Auswahl bestätigen',
  button_style='success'
)

# Ausgabe für die Dateiliste
file_list_output = widgets.Output()

# ---------- Funktionen ----------

def save_uploaded_files(change):
    for fileinfo in upload_widget.value:
        file_path = input_folder_parsed / fileinfo["name"]
        with open(file_path, 'wb') as f:
            f.write(fileinfo['content'])
    upload_widget.value = []
    update_file_list()

def update_file_list():
  with file_list_output:
    clear_output()
    mp4_files = list(input_folder_parsed.glob('*.mp4'))
    if mp4_files:
      display(HTML('<b>Vorhandene MP4-Dateien:</b>'))
      for file in mp4_files:
        display(HTML(f'📁 {file.name}'))
    else:
      display(HTML('<i>Keine MP4-Dateien im Ordner.</i>'))

def delete_all_mp4_files(b):
  for file in input_folder_parsed.glob('*.mp4'):
    file.unlink()
  update_file_list()

def confirm_selection(b):
  urls = url_textarea.value.strip().splitlines()
  print('✅ Bestätigt:')
  print(f'- {len(list(input_folder_parsed.glob("*.mp4")))} MP4-Dateien im Ordner')
  print(f'- {len(urls)} URLs eingegeben')
  for url in urls:
    print(f'  • {url.strip()}')
  with open(input_folder_parsed / 'input.txt', 'w') as f:
      f.write("\n".join(urls))

# ---------- Event-Handler verbinden ----------

upload_widget.observe(save_uploaded_files, names='value')
delete_button.on_click(delete_all_mp4_files)
confirm_button.on_click(confirm_selection)

# ---------- UI anzeigen ----------

display(widgets.VBox([
  upload_widget,
  url_textarea,
  delete_button,
  confirm_button,
  file_list_output
]))

# Initiale Anzeige
update_file_list()

## 2. Ausführen der Skelettschätzung
Die Skelettschätzungen laufen auf Docker-Containern. Mit einem Klick auf den Button starten Sie die Ausführung.

⚠ Das kann eine Weile dauern.

In [3]:
def run_container(service):
    print(f"⏳ Docker-Container läuft: {service}")
    container_name = f"{service}-{uuid.uuid4().hex[:8]}"
    result = subprocess.run(["docker", "compose", "run", "--no-deps", "--name", container_name, service])
    if result.returncode == 0:
        print(f"✔ Docker-Container beendet: {service}")
    else:
        print(f"❌ Fehler beim Ausführen von: {service} (Exit-Code: {result.returncode})")
        print(f"📜 Container-Logs von {container_name}:")
        subprocess.run(["docker", "logs", container_name])

@ray.remote
def run_container_as_service(poser):
    run_container(poser)

def run(posers=None):
    if posers is None:
        posers = get_services()
    subprocess.run(["docker", "compose", "build"])
    run_container("input-loader")
    tasks = [run_container_as_service.remote(poser) for poser in posers]
    ray.get(tasks)
    run_container("codecs")
    run_container("data-transformer")

def get_services(docker_compose_path="./docker-compose.yml"):
    with open(docker_compose_path) as f:
        dc = yaml.load(f, Loader=yaml.FullLoader)
    services = list(dc["services"].keys())
    services.remove("input-loader")
    services.remove("data-transformer")
    services.remove("codecs")
    return services

# Widgets
status_output = widgets.Output()

def confirm(b):
    with status_output:
        clear_output()
        display(HTML('<p style="color:orange;font-weight:bold;">⏳ Programm wird ausgeführt ...</p>'))
    run()
    with status_output:
        clear_output(wait=True)
        display(HTML('<p style="color:darkgreen;font-weight:bold;">✅ Ausführung beendet</p>'))

# Button
confirm_button = widgets.Button(
  description='Programm starten',
  button_style='success'
)
confirm_button.on_click(confirm)

# Anzeige
display(widgets.VBox([confirm_button, status_output]))

Statusmeldungen während der Ausführung der Container können Sie sich in den Logs ansehen:
`View > Show Log Console`